# Running MD simulations using nff and ASE

This Jupyter Notebook shows how the `nff` package interfaces with the Atomistic Simulation Environment (ASE). We assume the user went through tutorial `01_training`, so we can load the pretrained models without having to train them again.

As before, importing the dependencies:

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from ase import Atoms
from ase.md.verlet import VelocityVerlet

from nff.md.nve import Dynamics
from nff.data import Dataset
from nff.train import load_model, evaluate
import nff.utils.constants as const
from ase import units
from nff.io import NeuralFF, AtomsBatch

## Loading the relevant data

We reload the dataset and create a `GraphLoader` as we did last time:

In [2]:
dataset = Dataset.from_file('data/dataset.pth.tar')

### Creating Atoms

As before, we can create an `Atoms` object from any element of the dataset. Let's take the first one, for simplicity:

In [3]:
props = dataset[0].copy()
atoms = AtomsBatch(positions=props['nxyz'][:, 1:], 
                   numbers=props['nxyz'][:, 0], 
                   props=props,
                   )

### Creating the ASE calculator

Now we just have to load the ASE calculator from a pretrained model. One way of doing so is through the in-build `from_file` method. You just have to specify the folder where the model was trained and subsequently stored.

In [8]:
nff_ase = NeuralFF.from_file('sandbox/', device=3)

Assigning this calculator to `atoms` is easy:

In [9]:
atoms.set_calculator(nff_ase)

### Configuring the dynamics for the system

In this example, we will run an NVE dynamics simulation. We will use the default parameters there implemented to run a trajectory for an ethanol molecule. The parameters we will specify are the following:

* `T_init`: initial temperature of the simulation
* `time_step`: time step in femtoseconds
* `thermostat`: ASE integrator to use when performing the simulation
* `thermostat_params`: keyword arguments for ase.Integrator class, will be different case-by-case
* `steps`: number of steps to simulate
* `save_frequency`: how often (in steps) save the pose of the molecule in a file
* `nbr_list_update_freq`: how often (in steps) to update the neighbor list (not yet implemented)
* `thermo_filename`: output file for the thermodynamics log
* `traj_filename`: output file for the ASE trajectory file
* `skip`: number of initial frames to skip when recording the trajectory

In [10]:
md_params = {
    'T_init': 450,
    'time_step': 0.5,
#     'thermostat': NoseHoover,   # or Langevin or NPT or NVT or Thermodynamic Integration
#     'thermostat_params': {'timestep': 0.5 * units.fs, "temperature": 120.0 * units.kB,  "ttime": 20.0}
    'thermostat': VelocityVerlet,  
    'thermostat_params': {'timestep': 0.5 * units.fs},
    'steps': 200,
    'save_frequency': 10,
    'nbr_list_update_freq': 20,
    'thermo_filename': 'thermo.log',
    'traj_filename': 'atoms.traj',
    'skip': 0
}

In [11]:
nve = Dynamics(atoms, md_params)
nve.run()

Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]
0.0000           0.2068      -0.2971       0.5039   433.1

0.0050           0.2068      -0.1410       0.3478   299.0

0.0100           0.2067      -0.2650       0.4717   405.5

0.0150           0.2071      -0.3147       0.5218   448.6

0.0200           0.2066      -0.3889       0.5955   511.9

0.0250           0.2072      -0.2709       0.4781   411.0

0.0300           0.2072      -0.2095       0.4167   358.2

0.0350           0.2069      -0.3350       0.5419   465.8

0.0400           0.2083      -0.1967       0.4050   348.1

0.0450           0.2078      -0.2165       0.4243   364.8

0.0500           0.2078      -0.0776       0.2854   245.3

0.0550           0.2073      -0.3128       0.5200   447.0

0.0600           0.2068      -0.3093       0.5161   443.7

0.0650           0.2061      -0.2662       0.4723   406.0

0.0700           0.2075      -0.1981       0.4056   348.7

0.0750           0.2077      -0.3517       0.5594   480.9


### Models with directed neighbor lists

The default assumes that you're using SchNet, which uses an undirected neighbor list to save memory. If you're using Painn, DimeNet, or any model with directional information, you will need a directed neighbor list. If you don't specify this then you will get an error:

In [16]:
props = dataset[0].copy()
atoms = AtomsBatch(positions=props['nxyz'][:, 1:], 
                   numbers=props['nxyz'][:, 0], 
                   props=props
                   )
nff_ase = NeuralFF.from_file('sandbox_painn/', device=3)
atoms.set_calculator(nff_ase)

nve = Dynamics(atoms, md_params)
nve.run()

Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]


AssertionError: Painn needs a directed neighbor list

If you do then you will be fine!

In [17]:
props = dataset[0].copy()
atoms = AtomsBatch(positions=props['nxyz'][:, 1:], 
                   numbers=props['nxyz'][:, 0], 
                   props=props,
                   # specify directed
                   directed=True
                   )
nff_ase = NeuralFF.from_file('sandbox_painn/', device=3)
atoms.set_calculator(nff_ase)

nve = Dynamics(atoms, md_params)
nve.run()

Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]
0.0000           0.3057      -0.1615       0.4672   401.6

0.0050           0.3053      -0.1575       0.4628   397.8

0.0100           0.3048      -0.1632       0.4680   402.3

0.0150           0.3054      -0.1256       0.4310   370.5

0.0200           0.3049      -0.1621       0.4670   401.5

0.0250           0.3036      -0.1569       0.4605   395.8

0.0300           0.3047      -0.0502       0.3549   305.1

0.0350           0.3063      -0.0421       0.3484   299.4

0.0400           0.3062      -0.1249       0.4311   370.5

0.0450           0.3064      -0.0841       0.3905   335.7

0.0500           0.3049      -0.0987       0.4036   346.9

0.0550           0.3038      -0.1047       0.4085   351.2

0.0600           0.3025      -0.1648       0.4672   401.6

0.0650           0.3036      -0.2439       0.5475   470.6

0.0700           0.3063      -0.0606       0.3670   315.4

0.0750           0.3068      -0.0098       0.3167   272.2


The dynamics conserved the energy. The temperature varied throughout the simulation, as expected.

## Visualizing the trajectory

To visualize the trajectory in this Jupyter Notebook, you will have to install the package [nglview](https://github.com/arose/nglview).

In [ ]:
import nglview as nv
from ase.io import Trajectory

Displaying the trajectory:

In [ ]:
%matplotlib notebook

traj = Trajectory('atoms.traj')
nv.show_asetraj(traj)

Looks like the atoms are still together. Visual inspection says that the trajectory is reasonable. Yay for `nff`!